In [17]:
import pandas as pd
import sqlite3

In [18]:
base_iptu = pd.read_excel('dados/valores_sc.xlsx', )

In [19]:
base_iptu.columns.values[0] = 'setor'  # Renomeia a primeira coluna

In [20]:
base_iptu.head()

,setor,unidades,valor_terreno_medio,valor_construcao_medio,area_construida_media,area_terreno_media
0,355030801000001P,220,1954.426230,1921.486339,171.262295,188.732240
1,355030801000002P,838,1693.710145,1908.429952,369.101449,217.405797
2,355030801000003P,175,1794.658228,1922.240506,181.170886,205.917722
3,355030801000004P,130,1676.136364,1927.920455,198.147727,260.306818
4,355030801000005P,184,1929.859259,1912.992593,208.451852,260.622222


In [21]:
def carregar_dados_sqlite(arquivo_db, tabela):
    """
    Lê dados de uma tabela em um banco de dados SQLite e os transforma em um DataFrame do pandas.

    Parâmetros:
        arquivo_db (str): Caminho para o arquivo SQLite.
        tabela (str): Nome da tabela no banco de dados.

    Retorna:
        pd.DataFrame: DataFrame contendo os dados da tabela.
    """
    try:
        # Conexão com o banco de dados
        conexao = sqlite3.connect(arquivo_db)
        # Query para buscar todos os dados
        query = f"SELECT * FROM {tabela}"
        # Leitura dos dados em um DataFrame
        df = pd.read_sql_query(query, conexao)
        # Fechando a conexão
        conexao.close()
        return df
    except Exception as e:
        print(f"Erro ao carregar os dados: {e}")
        return None


In [22]:
arquivo_db = "dados_final.db"
tabela = "dados_final"
df_scraping = carregar_dados_sqlite(arquivo_db, tabela)


In [23]:
df_scraping.head()

,setor,preco_medio_m2,quantidade_imoveis,maior_preco_m2,menor_preco_m2,lat,lon
0,355030801000005P,4814.814815,1,4814.814815,4814.814815,-23.571521,-46.570585
1,355030801000074P,5175.438596,1,5175.438596,5175.438596,-23.577912,-46.572691
2,355030801000104P,9948.305448,6,12000.000000,8928.571429,-23.556107,-46.580224
3,355030801000131P,5672.713530,2,6530.612245,4814.814815,-23.570261,-46.571982
4,355030801000197P,9499.664333,14,11996.000000,8108.108108,-23.557752,-46.579734


In [24]:
def unificar_tabelas(df_sql, df_xlsx, coluna_setor):
    """
    Combina dois DataFrames pelo setor censitário e realiza um agrupamento.

    Parâmetros:
        df_sql (pd.DataFrame): DataFrame extraído do banco de dados SQL.
        df_xlsx (pd.DataFrame): DataFrame carregado do arquivo Excel.
        coluna_setor (str): Nome da coluna que representa o setor censitário.

    Retorna:
        pd.DataFrame: DataFrame unificado com informações agrupadas.
    """
    # Unificar as tabelas pelo setor censitário
    df_merged = pd.merge(df_sql, df_xlsx, on=coluna_setor, how="inner")
    
    # Realizar um agrupamento por setor, se necessário
    df_grouped = df_merged.groupby(coluna_setor).agg("mean").reset_index()
    
    return df_grouped

# Exemplo de uso
# Supondo que df_sql e df_xlsx sejam os DataFrames:
# df_unificado = unificar_tabelas(df_sql, df_xlsx, "setor_censitario")


In [25]:
df_unificado = unificar_tabelas(df_scraping, base_iptu, "setor")

In [28]:
df_unificado.head()

,setor,preco_medio_m2,quantidade_imoveis,maior_preco_m2,menor_preco_m2,lat,lon,unidades,valor_terreno_medio,valor_construcao_medio,area_construida_media,area_terreno_media
0,355030801000005P,4814.814815,1.0,4814.814815,4814.814815,-23.571521,-46.570585,184.0,1929.859259,1912.992593,208.451852,260.622222
1,355030801000074P,5175.438596,1.0,5175.438596,5175.438596,-23.577912,-46.572691,179.0,1707.290698,1954.627907,159.348837,216.244186
2,355030801000104P,9948.305448,6.0,12000.000000,8928.571429,-23.556107,-46.580224,266.0,2262.575758,1945.575758,239.903030,195.509091
3,355030801000131P,5672.713530,2.0,6530.612245,4814.814815,-23.570261,-46.571982,359.0,1829.363128,1908.234637,256.513966,207.720670
4,355030801000197P,9499.664333,14.0,11996.000000,8108.108108,-23.557752,-46.579734,132.0,2416.809524,1937.104762,169.076190,145.676190


In [27]:
df_unificado.shape

(1258, 12)

In [29]:
df_unificado.rename(columns={'preco_medio_m2': 'preco_medio_m2_qa', 'maior_preco_m2':'maior_preco_qa',
                             'menor_preco_m2': 'menor_preco_qa', 'valor_terreno_medio':'preco_medio_m2_terreno_iptu',
                             'valor_construcao_medio':'preco_medio_m2_construcao_iptu'}, inplace=True)

In [30]:
df_unificado.head()

,setor,preco_medio_m2_qa,quantidade_imoveis,maior_preco_qa,menor_preco_qa,lat,lon,unidades,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,area_construida_media,area_terreno_media
0,355030801000005P,4814.814815,1.0,4814.814815,4814.814815,-23.571521,-46.570585,184.0,1929.859259,1912.992593,208.451852,260.622222
1,355030801000074P,5175.438596,1.0,5175.438596,5175.438596,-23.577912,-46.572691,179.0,1707.290698,1954.627907,159.348837,216.244186
2,355030801000104P,9948.305448,6.0,12000.000000,8928.571429,-23.556107,-46.580224,266.0,2262.575758,1945.575758,239.903030,195.509091
3,355030801000131P,5672.713530,2.0,6530.612245,4814.814815,-23.570261,-46.571982,359.0,1829.363128,1908.234637,256.513966,207.720670
4,355030801000197P,9499.664333,14.0,11996.000000,8108.108108,-23.557752,-46.579734,132.0,2416.809524,1937.104762,169.076190,145.676190


In [32]:
df_teste = df_unificado[['setor', 'preco_medio_m2_qa', 'preco_medio_m2_terreno_iptu', 'preco_medio_m2_construcao_iptu', 'lat', 'lon']]

In [33]:
df_teste.head()

,setor,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734


In [34]:
df_teste['diferenca_qa_iptu_terreno'] = df_teste['preco_medio_m2_qa'] - df_teste['preco_medio_m2_terreno_iptu']

C:\Users\ggsna\AppData\Local\Temp\ipykernel_26612\203411054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['diferenca_qa_iptu_terreno'] = df_teste['preco_medio_m2_qa'] - df_teste['preco_medio_m2_terreno_iptu']


In [35]:
df_teste.head()

,setor,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon,diferenca_qa_iptu_terreno
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585,2884.955556
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691,3468.147899
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224,7685.729691
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982,3843.350401
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734,7082.854809


In [36]:
df_teste['diff_percentual'] = ( df_teste['preco_medio_m2_qa'] / df_teste['diferenca_qa_iptu_terreno']) * 100

C:\Users\ggsna\AppData\Local\Temp\ipykernel_26612\1655758279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste['diff_percentual'] = ( df_teste['preco_medio_m2_qa'] / df_teste['diferenca_qa_iptu_terreno']) * 100


In [37]:
df_teste.head()

,setor,preco_medio_m2_qa,preco_medio_m2_terreno_iptu,preco_medio_m2_construcao_iptu,lat,lon,diferenca_qa_iptu_terreno,diff_percentual
0,355030801000005P,4814.814815,1929.859259,1912.992593,-23.571521,-46.570585,2884.955556,166.893899
1,355030801000074P,5175.438596,1707.290698,1954.627907,-23.577912,-46.572691,3468.147899,149.227736
2,355030801000104P,9948.305448,2262.575758,1945.575758,-23.556107,-46.580224,7685.729691,129.438659
3,355030801000131P,5672.713530,1829.363128,1908.234637,-23.570261,-46.571982,3843.350401,147.598135
4,355030801000197P,9499.664333,2416.809524,1937.104762,-23.557752,-46.579734,7082.854809,134.121969
